## 1.d Micro-level Water demand model

In [1]:
import datetime; print(datetime.datetime.now())

2017-10-25 14:39:45.067213


**Notebook abstract**

A simple micro-level water demand model.

A simple micro-level water demand model. Similar to the [income demand model](Ab_MCMC_income.ipynb) and the [electricity demand model](Ac_MCMC_electricity.ipynb), the water demand model uses micro level consumption demand data for the construction of a table model. The table model describes simple rules for the construction of the proxy micro level sample data.  

### Prior water demand model

In [2]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
from urbanmetabolism._scripts.micro import compute_categories, change_index

/usr/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
water_data = pd.read_csv('data/water.csv', index_col=0)
formula = "Water_expenditure ~ Total_Family_Income + Family_Size + C(HH_head_Sex)\
+ HH_head_Age + C(Education) + C(Urbanity)"

In [4]:
water_data.head()

,Family_Size,HH_head_Sex,HH_head_Age,Education,Electricity_expenditure,Water_expenditure,Total_Family_Income,Urbanity
3,2.0,2,51,1.0,900,2190,9932.333333,0
9,3.5,1,41,4.0,17202,300,47233.833333,0
11,3.0,2,75,1.0,4212,3024,16521.333333,0
12,4.5,1,74,1.0,6210,4086,20254.333333,0
13,6.5,2,55,2.0,3900,2940,16368.000000,0


In [5]:
model_water = sm.WLS.from_formula(formula, water_data)
model_results_water = model_water.fit()

In [6]:
model_results_water.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:      Water_expenditure   R-squared:                       0.344
Model:                            WLS   Adj. R-squared:                  0.344
Method:                 Least Squares   F-statistic:                     925.6
Date:                Mon, 23 Oct 2017   Prob (F-statistic):               0.00
Time:                        18:02:49   Log-Likelihood:            -1.3853e+05
No. Observations:               15904   AIC:                         2.771e+05
Df Residuals:                   15894   BIC:                         2.771e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept            -601.5920     62.632     -9.605      0.000    -724.358    -478.826
C(HH_head_Sex)[T.2]    98.4950     29.444      3.345      0.001      40.782     156.208
C(Education)[T.2.0]   214.4011     28.816      7.440      0.000     157.919     270.883
C(Education)[T.3.0]   260.3273     40.057      6.499      0.000     181.810     338.844
C(Education)[T.4.0]   101.7028     49.996      2.034      0.042       3.705     199.700
C(Education)[T.5.0]    40.1879    119.681      0.336      0.737    -194.400     274.775
C(Urbanity)[T.1]     1000.9789     25.416     39.384      0.000     951.161    1050.797
Total_Family_Income     0.0532      0.001     54.145      0.000       0.051       0.055
Family_Size            49.7394      5.898      8.434      0.000      38.179      61.300
HH_head_Age             6.0889      0.913      6.671      0.000       4.300       7.878
==============================================================================
Omnibus:                     3226.790   Durbin-Watson:                   1.399
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7597.976
Skew:                           1.142   Prob(JB):                         0.00
Kurtosis:                       5.499   Cond. No.                     3.10e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.1e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [7]:
params_water = change_index(model_results_water.params)
bse_water = change_index(model_results_water.bse)
water = pd.concat([params_water, bse_water], axis=1)
water.columns = ['co_mu', 'co_sd']
water = compute_categories(water)

In [8]:
water.loc['Urbanity', 'p'] = (water_data.Urbanity == 1).sum() / water_data.shape[0]
water.loc['Sex', 'p'] = (water_data.HH_head_Sex == 2).sum() / water_data.shape[0]

In [9]:
water.loc[:,'dis'] = 'None'
water.loc['Education', 'dis'] = 'None;i;Categorical'
water.loc['Intercept', 'dis'] = 'Deterministic'

In [10]:
water.loc[:, 'mu'] = np.nan
water.loc[:, 'sd'] = np.nan
water.loc['Intercept', 'p'] = water.loc['Intercept', 'co_mu']
water.loc['Intercept', ['co_mu', 'co_sd']] = np.nan

In [11]:
water.loc[:,'ub'] = np.nan
water.loc[:,'lb'] = np.nan

In [12]:
water.index = ['w_'+i for i in water.index]

In [13]:
water.to_csv('data/table_water.csv')

In [14]:
water

,co_mu,co_sd,p,dis,mu,sd,ub,lb
w_Intercept,NaN,NaN,-601.591950,Deterministic,NaN,NaN,NaN,NaN
w_Sex,98.495,29.4438,0.224786,None,NaN,NaN,NaN,NaN
w_Urbanity,1000.98,25.4159,0.593939,None,NaN,NaN,NaN,NaN
w_Total_Family_Income,0.053187,0.000982306,NaN,None,NaN,NaN,NaN,NaN
w_FamilySize,49.7394,5.89779,NaN,None,NaN,NaN,NaN,NaN
w_Age,6.08894,0.912741,NaN,None,NaN,NaN,NaN,NaN
w_Education,"1.0,214.401145313,260.327274277,101.70283943,4...","1e-10,28.8158024405,40.0574490885,49.995759305...",NaN,None;i;Categorical,NaN,NaN,NaN,NaN


-----

[UN Environment](http://www.unep.org/)

[Home](Welcome.ipynb)

[Next](Ae_MCMC_nonres.ipynb) (1.e) Micro-level Non-Residential model